# Notebook com as funções de otimização de carteiras:


*   CVM (Carteira de Variância Mínima)
*   Paridade de Risco



## Definição de Installs

## Definição de imports

In [ ]:
# imports de bibliotecas
import numpy as np
import pandas as pd
import math
import random
from scipy.optimize import minimize


## Definições de variáveis globais

## Função para implementar a estratégia CVM (Carteira de Variância Mínima)

In [ ]:
def CVM(df_portfolio, df_matrix_covarianvcia, df_retorno_acumulado_por_ativo):

  cons = {'type': 'eq', 'fun': lambda pesos: pesos.sum() - 1.0}
  qtd_ativos = len(df_portfolio.columns)

  bnds = qtd_ativos * [(0.01, 1),]

  peso_inicial_ativo = 1/qtd_ativos

  pesos_iniciais_ativos = [peso_inicial_ativo] * int(qtd_ativos)

  opt = minimize(lambda p: calcular_desvio_padrao(p, df_portfolio, df_matrix_covarianvcia), pesos_iniciais_ativos, bounds=bnds, constraints=cons)

  pesos_otimizados_ativos = opt['x']

  df_rp = pd.DataFrame(pesos_otimizados_ativos).transpose()

  df_rp.columns = df_portfolio.columns

  df_rp = df_rp.transpose()

  df_rp.sort_values(by=0, ascending=False, inplace=True)

  return df_rp, opt['fun']

In [ ]:
def calcular_desvio_padrao(pesos,df_portfolio, df_matrix_covarianvcia):

  array_ones = np.ones(len(df_portfolio.columns))

  quantidade_ativos = array_ones.sum()

  dt_pesos_iniciais_ativos = pd.DataFrame(pesos)

  covarianca_x_pesos = np.matmul(df_matrix_covarianvcia,pesos)

  varianca = np.sum(pesos*covarianca_x_pesos)

  volatilidade = varianca.sum()**0.5

  volatilidade = abs(volatilidade)


  return volatilidade.sum()


## Função para implementar a estratégia Paridade de Risco

In [ ]:
def PR(df_portfolio, df_matrix_covarianvcia):

  cons = {'type': 'eq', 'fun': lambda pesos: pesos.sum() - 1.0}
  qtd_ativos = len(df_portfolio.columns)

  bnds = qtd_ativos * [(0.01, 1),]

  peso_inicial_ativo = 1/qtd_ativos

  pesos_iniciais_ativos = [peso_inicial_ativo] * int(qtd_ativos)

  opt = minimize(lambda p: calculo_desvio(p, df_portfolio, df_matrix_covarianvcia), pesos_iniciais_ativos, bounds=bnds, constraints=cons)

  pesos_otimizados_ativos = opt['x']

  df_rp = pd.DataFrame(pesos_otimizados_ativos).transpose()

  df_rp.columns = df_portfolio.columns

  df_rp = df_rp.transpose()

  df_rp.sort_values(by=0, ascending=False, inplace=True)

  return df_rp, opt['fun']

### Função para cálculo do desvio

In [ ]:
def calculo_desvio(pesos,df_portfolio, df_matrix_covarianvcia):

  array_ones = np.ones(len(df_portfolio.columns))

  quantidade_ativos = array_ones.sum()

  dt_pesos_iniciais_ativos = pd.DataFrame(pesos)

  covarianca_x_pesos = np.matmul(df_matrix_covarianvcia,pesos)

  varianca = np.sum(pesos*covarianca_x_pesos)

  volatilidade = varianca.sum()**0.5

  pesos_por_varianca = pesos/varianca

  contribuicao_risco = covarianca_x_pesos*pesos_por_varianca

  desvio = contribuicao_risco-(1/quantidade_ativos)

  desvio = abs(desvio)


  return desvio.sum()

## Função para implementar a estratégia Naïve

In [ ]:
def Naive(Funcoes_Apoio,df_portfolio, df_retorno_acumulado_por_ativo):
  qtd_ativos = len(df_portfolio.columns)

  peso_inicial_ativo = 1/qtd_ativos

  dict = {'Pesos':[peso_inicial_ativo]*qtd_ativos
       }

  pesos_df = pd.DataFrame(dict)

  df_retornos = pd.DataFrame(df_retorno_acumulado_por_ativo)

  pesos_df.index = df_retornos.index

  return Funcoes_Apoio.RetornoPortfolio(pesos_df.transpose(),df_retornos)